In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models # hazır modeller için(models)
from torchvision.utils import make_grid
import os
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
torch.manual_seed(42)
# uyarıları yok say
import warnings
warnings.filterwarnings("ignore")


train_transform = transforms.Compose([
    transforms.Resize((224,224)), # yeniden boyutlandırma
    transforms.ToTensor(),
    transforms.Normalize([0.1943, 0.5623, 0.6537],[0.1747, 0.2768, 0.3338])

])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.1943, 0.5623, 0.6537],[0.1747, 0.2768, 0.3338])
    
])


data = 'vks'


#görsellerin okunması ve düzenleme işleminin uygulanması
train_data = datasets.ImageFolder(os.path.join(data, 'train'), transform=train_transform)
test_data = datasets.ImageFolder(os.path.join(data, 'test'), transform=test_transform)

torch.manual_seed(42)
train_loader = DataLoader(train_data, batch_size=5, shuffle=True)
test_loader = DataLoader(test_data, batch_size=5, shuffle=True)

class_names = train_data.classes #sınıflar

vgg16model = models.vgg16(pretrained=False)

for param in vgg16model.parameters():
    param.requires_grad = False


vgg11model.classifier = nn.Sequential(nn.Linear(25088, 4096),
                                 nn.ReLU(),
                                 nn.Dropout(0.8),
                                 nn.Linear(4096, 5),
                                 nn.LogSoftmax(dim=1))


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vgg16model.classifier.parameters(), lr=0.0001)

import time
start_time = time.time()

epochs = 10

max_trn_batch = 5
max_tst_batch = 1

train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    trn_corr = 0
    tst_corr = 0
    
    
    for b, (X_train, y_train) in enumerate(train_loader):
        if b == max_trn_batch:
            break
        b+=1
        
        
        y_pred = vgg16model(X_train)
        loss = criterion(y_pred, y_train)
 
        
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
        if b%10 == 0:
            print(f'epoch: {i:2}  batch: {b:4} [{10*b:6}/100]  loss: {loss.item():10.8f}  \
accuracy: {trn_corr.item()*100/(10*b):7.3f}%')

    train_losses.append(loss)
    train_correct.append(trn_corr)

    
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):
            if b == max_tst_batch:
                break

           
            y_val = vgg16model(X_test)

           
            predicted = torch.max(y_val.data, 1)[1] 
            tst_corr += (predicted == y_test).sum()

    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed